In [2]:
!pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 ultralytics python-doctr PyMuPDF pillow einops numpy transformers peft diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 852.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 842.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━

In [3]:
from ultralytics import YOLO
import fitz
import os
import pathlib
from PIL import Image, ImageEnhance
import numpy as np
import os
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
!wget -q https://huggingface.co/DILHTWD/documentlayoutsegmentation_YOLOv8_ondoclaynet/resolve/main/yolov8x-doclaynet-epoch64-imgsz640-initiallr1e-4-finallr1e-5.pt

In [7]:
# List of sample PDF files to process
pdf_list = ['test.pdf']
# Load the document segmentation model
docseg_model = YOLO('yolov8x-doclaynet-epoch64-imgsz640-initiallr1e-4-finallr1e-5.pt')
# Initialize a dictionary to store results
mydict = {}

def enhance_image(img):
    """Apply image enhancements for better quality."""
    # Enhance sharpness
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(1.5)

    # Enhance contrast
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(1.2)

    # Enhance color
    enhancer = ImageEnhance.Color(img)
    img = enhancer.enhance(1.1)

    return img

def process_pdf_page(pdf_path, page_num, docseg_model, output_dir):
    """Processes a single page of a PDF with maximum quality settings."""

    pdf_doc = fitz.open(pdf_path)
    page = pdf_doc[page_num]

     # Increase the resolution matrix for maximum quality
    zoom = 4  # Increased zoom factor for higher resolution
    matrix = fitz.Matrix(zoom, zoom)

    # Use high-quality rendering options
    pix = page.get_pixmap(
        matrix=matrix,
        alpha=False,  # Disable alpha channel for clearer images
        colorspace=fitz.csRGB  # Force RGB colorspace
    )

    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Apply image enhancements
    img = enhance_image(img)

    # Resize with high-quality settings
    if zoom != 1:
        original_size = (int(page.rect.width), int(page.rect.height))
        img = img.resize(original_size, Image.Resampling.LANCZOS)
    # Generate a temporary filename for the page image
    temp_img_filename = os.path.join(output_dir, f"temp_page_{page_num}.png")

    # Save with maximum quality settings
    img.save(
        temp_img_filename,
        "PNG",
        quality=100,
        optimize=False,
        dpi=(300, 300)  # Set high DPI
    )
    # Run the model on the image
    results = docseg_model(source=temp_img_filename, save=True, show_labels=True, show_conf=True, boxes=True)
    # Extract the results
    page_width = page.rect.width
    one_third_width = page_width / 3


    all_coords = []

    for entry in results:
        thepath = pathlib.Path(entry.path)
        thecoords = entry.boxes.xyxy.numpy()
        all_coords.extend(thecoords)

    # Sort the coordinates into two groups and then sort each group by y1
    left_group = []
    right_group = []
    for bbox in all_coords:
            x1 = bbox[0]
            if x1 < one_third_width:
                left_group.append(bbox)
            else:
                right_group.append(bbox)

    left_group = sorted(left_group, key=lambda bbox: bbox[1])
    right_group = sorted(right_group, key=lambda bbox: bbox[1])

    sorted_coords = left_group + right_group

    mydict[f"{pdf_path} Page {page_num}"] = sorted_coords
    # Clean up the temporary image
    os.remove(temp_img_filename)
    pdf_doc.close()


# Process each PDF in the list
for pdf_path in pdf_list:
    try:
        pdf_doc = fitz.open(pdf_path)
        num_pages = pdf_doc.page_count
        pdf_doc.close()
        output_dir = os.path.splitext(pdf_path)[0] + "_output"
        os.makedirs(output_dir, exist_ok=True)
        for page_num in range(num_pages):
            process_pdf_page(pdf_path, page_num, docseg_model, output_dir)
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
# Create the 'tmp' directory if it doesn't exist
tmp_dir = 'tmp'
os.makedirs(tmp_dir, exist_ok=True)
# Iterate through the results and save cropped images with maximum quality
for key, coords in mydict.items():
    pdf_name, page_info = key.split(" Page ")
    page_number = int(page_info)
    pdf_doc = fitz.open(pdf_name)
    page = pdf_doc[page_number]

    zoom = 4
    matrix = fitz.Matrix(zoom,zoom)
    for i, bbox in enumerate(coords):
        # Scale the bounding box coordinates appropriately
        xmin, ymin, xmax, ymax = map(lambda x: x , bbox)

        # Create a rectangle from the bounding box
        rect = fitz.Rect(xmin, ymin, xmax, ymax)

        # Crop using get_pixmap with a maximum resolution matrix
        cropped_pix = page.get_pixmap(
            clip=rect,
            matrix=matrix,
            alpha=False,
            colorspace=fitz.csRGB
        )

        cropped_img = Image.frombytes("RGB", [cropped_pix.width, cropped_pix.height], cropped_pix.samples)
        cropped_img = enhance_image(cropped_img)

        output_filename = os.path.join(tmp_dir, f"{os.path.splitext(os.path.basename(pdf_name))[0]}_page{page_number}_{i}.png")

        # Save the cropped image
        cropped_img.save(output_filename, "PNG", quality=100, optimize=False, dpi=(300, 300))
    pdf_doc.close()

def extract_text_from_image(image_path, model):
    """Extracts text from a single image using DocTr."""
    doc = DocumentFile.from_images(image_path)
    result = model(doc)
    text_content = ""
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                for word in line.words:
                    text_content += word.value + " "
            text_content += "\n"
    return text_content.strip()

def process_cropped_images(tmp_dir, pdf_list):
    """Iterates through cropped images, extracts text using DocTr and stores the text in text files."""

    doctr_model = ocr_predictor(pretrained=True)

    for pdf_path in pdf_list:
        pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
        output_txt_path = f"{pdf_name}_extracted_text.txt"

        with open(output_txt_path, 'w', encoding='utf-8') as outfile:

            pdf_doc = fitz.open(pdf_path)
            num_pages = pdf_doc.page_count
            pdf_doc.close()
            for page_num in range(num_pages):

                outfile.write(f"Page: {page_num}\n")

                # Sort filenames of cropped images by chunk order
                cropped_images_for_page = sorted([
                    f for f in os.listdir(tmp_dir)
                    if f.startswith(f"{pdf_name}_page{page_num}_") and f.endswith(".png")
                ], key=lambda f: int(f.split("_")[-1].split(".")[0]))

                for i, image_filename in enumerate(cropped_images_for_page):
                    image_path = os.path.join(tmp_dir, image_filename)
                    text = extract_text_from_image(image_path, doctr_model)
                    outfile.write(f"  Chunk {i}: {text}\n")
        print(f"Text extracted from {pdf_name} saved to {output_txt_path}")

# Example usage:
tmp_dir = 'tmp' # Make sure your tmp directory exists
pdf_list = ['test.pdf'] # Your list of PDFs
process_cropped_images(tmp_dir, pdf_list)

WARNING ⚠️ 'boxes' is deprecated and will be removed in the future. Use 'show_boxes' instead.

image 1/1 /content/test_output/temp_page_0.png: 640x512 1 Caption, 1 List-item, 1 Page-footer, 2 Pictures, 1 Section-header, 11 Texts, 4344.4ms
Speed: 4.3ms preprocess, 4344.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 512)
Results saved to /content/runs/detect/predict2
WARNING ⚠️ 'boxes' is deprecated and will be removed in the future. Use 'show_boxes' instead.

image 1/1 /content/test_output/temp_page_1.png: 640x512 6 List-items, 1 Page-footer, 2 Pictures, 1 Section-header, 3395.6ms
Speed: 4.0ms preprocess, 3395.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 512)
Results saved to /content/runs/detect/predict2
Text extracted from test saved to test_extracted_text.txt
